# Tourist Route Planning with Genetic Algorithm - Comprehensive Implementation

## Overview
This notebook implements and compares three optimization algorithms for Tourist Route Planning with Time Windows and Budget Constraints (TRPTW):

1. **Enhanced Genetic Algorithm** - Primary optimization method
2. **Greedy Algorithm** - Fast heuristic approach
3. **Random Search** - Baseline comparison

### Research Context
Based on research by Dai Thanh Long Doan et al. on optimizing tourist route planning using genetic algorithms with practical constraints including attraction time windows and tourist budgets.

In [1]:
# Import required libraries
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import json
import warnings
warnings.filterwarnings('ignore')

# Set style and random seeds
plt.style.use('seaborn-v0_8')
np.random.seed(42)
random.seed(42)

print("🚀 Tourist Route Planning System Initialized")
print("📊 Ready for comprehensive algorithm comparison")

Matplotlib is building the font cache; this may take a moment.


🚀 Tourist Route Planning System Initialized
📊 Ready for comprehensive algorithm comparison


## Utility Functions

In [2]:
def euclidean(a, b):
    """Calculate Euclidean distance between two points"""
    return math.hypot(a[0] - b[0], a[1] - b[1])

def minutes_to_time_str(minutes):
    """Convert minutes from 9:00 AM base to time string"""
    base_hour = 9
    h = base_hour + minutes // 60
    m = minutes % 60
    return f"{int(h):02d}:{int(m):02d}"

def time_str_to_minutes(time_str):
    """Convert time string to minutes from 9:00 AM base"""
    h, m = map(int, time_str.split(':'))
    return (h - 9) * 60 + m

## Dataset Generation

Generate enhanced datasets for three Vietnamese cities with realistic attraction characteristics.

In [3]:
def generate_attractions_dataset(city_name, num_attractions, grid_size=3.0):
    """Generate a comprehensive dataset of attractions for testing"""
    attractions = []
    
    # Predefined attraction types with realistic characteristics
    attraction_types = {
        'Museum': {'base_duration': 75, 'base_cost': 35000, 'open_early': True},
        'Temple': {'base_duration': 40, 'base_cost': 10000, 'open_early': True},
        'Park': {'base_duration': 60, 'base_cost': 0, 'open_early': False},
        'Market': {'base_duration': 45, 'base_cost': 5000, 'open_early': True},
        'Monument': {'base_duration': 30, 'base_cost': 15000, 'open_early': True},
        'Beach': {'base_duration': 90, 'base_cost': 0, 'open_early': False},
        'Shopping': {'base_duration': 80, 'base_cost': 20000, 'open_early': True},
        'Cultural Site': {'base_duration': 55, 'base_cost': 25000, 'open_early': True},
    }
    
    type_names = list(attraction_types.keys())
    
    for i in range(1, num_attractions + 1):
        # Random coordinates within grid
        x = random.uniform(-grid_size/2, grid_size/2)
        y = random.uniform(-grid_size/2, grid_size/2)
        
        # Select attraction type
        attr_type = random.choice(type_names)
        type_data = attraction_types[attr_type]
        
        # Generate opening/closing times based on type
        if type_data['open_early']:
            open_hour = random.choice([6, 7, 8])
            close_hour = random.choice([17, 18, 19])
        else:
            open_hour = random.choice([8, 9, 10])
            close_hour = random.choice([19, 20, 21, 22])
            
        # Some attractions are 24/7
        if attr_type in ['Park', 'Beach'] and random.random() < 0.3:
            open_time = "00:00"
            close_time = "23:59"
        else:
            open_time = f"{open_hour:02d}:{random.choice([0, 30]):02d}"
            close_time = f"{close_hour:02d}:{random.choice([0, 30]):02d}"
        
        # Duration and cost with variation
        duration = type_data['base_duration'] + random.randint(-15, 20)
        cost = type_data['base_cost'] + random.randint(-5000, 10000)
        cost = max(0, cost)  # Ensure non-negative
        
        attraction = {
            'id': i,
            'name': f"{city_name} {attr_type} {i}",
            'coord': (x, y),
            'open': open_time,
            'close': close_time,
            'duration': duration,
            'cost': cost
        }
        attractions.append(attraction)
    
    return attractions

def make_df(attractions):
    """Convert attractions list to DataFrame with time in minutes from 9:00 AM"""
    rows = []
    for a in attractions:
        oh, om = map(int, a['open'].split(':'))
        ch, cm = map(int, a['close'].split(':'))
        open_min = (oh - 9) * 60 + om
        close_min = (ch - 9) * 60 + cm
        
        rows.append({
            'id': a['id'], 
            'name': a['name'], 
            'x': a['coord'][0], 
            'y': a['coord'][1],
            'open_min': open_min, 
            'close_min': close_min, 
            'duration': a['duration'], 
            'cost': a['cost']
        })
    return pd.DataFrame(rows)

In [4]:
# Generate datasets for three Vietnamese cities
print("🏙️ Generating enhanced datasets...")

# Create datasets with varying complexity
hanoi_attractions = generate_attractions_dataset("Hanoi", 20, grid_size=4.0)
danang_attractions = generate_attractions_dataset("Da Nang", 15, grid_size=3.5)
hcmc_attractions = generate_attractions_dataset("HCMC", 25, grid_size=5.0)

# Convert to DataFrames
hanoi_df = make_df(hanoi_attractions)
danang_df = make_df(danang_attractions)
hcmc_df = make_df(hcmc_attractions)

# Display dataset summaries
datasets = {
    'Hanoi': hanoi_df,
    'Da Nang': danang_df,
    'Ho Chi Minh City': hcmc_df
}

print(f"Dataset Summary:")
for city, df in datasets.items():
    print(f"  {city}: {len(df)} attractions")
    print(f"    Cost range: {df['cost'].min():,} - {df['cost'].max():,} VND")
    print(f"    Duration range: {df['duration'].min()} - {df['duration'].max()} minutes")
    print()

# Display first few rows
print("Sample data (Hanoi):")
hanoi_df.head()

🏙️ Generating enhanced datasets...
Dataset Summary:
  Hanoi: 20 attractions
    Cost range: 0 - 44,952 VND
    Duration range: 20 - 93 minutes

  Da Nang: 15 attractions
    Cost range: 2,179 - 41,062 VND
    Duration range: 27 - 90 minutes

  Ho Chi Minh City: 25 attractions
    Cost range: 0 - 42,781 VND
    Duration range: 16 - 93 minutes

Sample data (Hanoi):


,id,name,x,y,open_min,close_min,duration,cost
0,1,Hanoi Monument 1,0.557707,-1.899957,-180,480,49,11424
1,2,Hanoi Temple 2,0.361970,-1.872869,-180,480,59,11873
2,3,Hanoi Museum 3,-1.118238,0.357063,-150,630,77,32547
3,4,Hanoi Temple 4,-1.138745,1.053977,-180,570,47,14891
4,5,Hanoi Cultural Site 5,-0.941917,-1.826198,-30,480,75,24803


In [5]:
df

,id,name,x,y,open_min,close_min,duration,cost
0,1,HCMC Park 1,-1.922857,-1.964820,30,720,61,3280
1,2,HCMC Museum 2,-0.057341,2.026682,-150,630,62,30058
2,3,HCMC Monument 3,-0.832217,-1.845902,-150,630,50,10158
3,4,HCMC Park 4,-1.940631,2.225255,60,630,53,0
4,5,HCMC Museum 5,-0.958708,1.994907,-120,480,82,42781
5,6,HCMC Park 6,0.299418,1.873564,-60,690,46,0
6,7,HCMC Shopping 7,1.182822,-0.839073,-60,630,75,27900
7,8,HCMC Museum 8,1.006391,-0.587291,-120,510,82,35000
8,9,HCMC Market 9,1.602682,1.856615,-180,630,51,4564
9,10,HCMC Monument 10,1.821765,2.334446,-90,630,16,11889
